# Purpose

- Download a dataset with wind turbine locations in Denmark

### Input files
 - None

### Outputs files
- `turbines.geojson` - Location of all wind turbines in Denmark as EPSG:4326 (Web mercator)
- `offshore_turbines.json` - Location of offshore wind turbines in Denmark as EPSG:4326 (Web mercator)

In [1]:
# File processing libraries
import zipfile
import pathlib
    
# Data processing libraries
import pandas as pd

# Geospatial libraries
import geopandas as gpd
import shapely
import pyproj
#import utm

# Network libraries
from urllib.request import urlretrieve

/home/christian/.local/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
wind_master_data_location = "https://ens.dk/sites/ens.dk/files/Analyser/stamdata_eksisterende_moeller_2023_01_etrs1989_utm32n.zip"
wind_master_data_filename = "stamdata_eksisterende_moeller_2023_01_etrs1989_utm32n.zip"
wind_master_data_folder = wind_master_data_filename.split('.')[0]

if not pathlib.Path(wind_master_data_filename).exists():
    # Download file
    urlretrieve(wind_master_data_location, wind_master_data_filename)
    
    # Extract file
    with zipfile.ZipFile(wind_master_data_filename, 'r') as zip_ref:
        zip_ref.extractall('.')

In [3]:
# Load data
turbines = gpd.read_file(wind_master_data_folder)

In [4]:
# Convert geometry from UTM 32N to Lat/Lng
wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:25832')
project = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
new_geo = turbines.apply(lambda row: shapely.ops.transform(project, row.geometry), axis=1)

/home/christian/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
# Create new column with lat/lng
turbines['lng'] = new_geo.geometry.x
turbines['lat'] = new_geo.geometry.y

# Remove columns that are not interesting to save disk space
turbines = turbines[['GSRN', 'Kapacitet_', 'Rotordiame', 'Navhøjde_', 'Fabrikat', 'Y2022_10', 'Y2022_11', 'Y2022_12', 'lat', 'lng', 'geometry']]

In [6]:
turbines.to_file('turbines.geojson', driver="GeoJSON")  

## Extract only offshore turbines

In [7]:
# Get offshore locations
geo_sites = gpd.read_file("../../NoCode/data/dk_offshore_sites.geojson")

In [8]:
# Perform spatial join to only turbines within area
offshore_turbines = geo_sites.sjoin(turbines.to_crs("EPSG:4326"), how='left')

In [9]:
# Also export as csv
offshore_turbines[['Kapacitet_', 'Rotordiame', 'Navhøjde_', 'lat', 'lng']].to_json('offshore_turbines.json', orient='records')